##  Implementing the custom cost function with ruptures format

In [1]:
from Detection_method import *
from metrics import *
from Data_op import *
from os import listdir
from sklearn import preprocessing
class MyCost(BaseCost):

    """Custom cost for exponential signals."""

    # The 2 following attributes must be specified for compatibility.
    model = "simp"
    min_size = 2

    def fit(self, signal):
        """Set the internal parameter."""
        self.signal = signal
        self.cumsum = np.cumsum(signal,axis=0)
        cumsum1=[0]
        for i in range(signal.shape[0]):
            y=signal[i].reshape((signal.shape[1],1))
            cumsum1.append(cumsum1[i]+y.dot(np.transpose(y)))
        self.cumsum1=cumsum1
            
            
        return self

    def error(self, start, end):
        sub = self.signal[start:end,:]
        mean=1/len(sub)*(self.cumsum[end-2,:]-self.cumsum[max(0,start-1),:])
        term1=1/len(sub)*(self.cumsum1[end-1]-self.cumsum1[start-1])
        term2=mean.dot(np.transpose(mean))
        var=term1-term2
        #var = pd.DataFrame(sub).cov()
        cost=(end-start)*np.linalg.slogdet(var)[1]
        return cost

## Running the process on multiple signals

In [11]:
def process_data(link,method,cost,side):
    print("initiating process for: ",link)
    data=get_side_data(data_acquisition(link),side)
    meta_data=data[1]

    signals=np.asarray(data[0])
    steps=meta_data
    bkps=np.asarray(steps)
    bkps=bkps.reshape(bkps.shape[1]*bkps.shape[0])
    algo = method(custom_cost=cost).fit(signals)
    my_bkps = np.asarray(algo.predict(n_bkps=len(bkps)+1))[1:]
    segs=[]
    detected_steps=[]
    d=pd.DataFrame(preprocessing.scale(data[0]))
    for i in range(len(my_bkps)-1):
        segs.append((my_bkps[i],my_bkps[i+1]))
    limit=np.linalg.norm(d.std())
    for s in segs:
        chunk=d.iloc[s[0]:s[1],:]
        x=pd.DataFrame(preprocessing.scale(chunk,with_std=False))
        if(np.linalg.norm(x.std())>limit):
            detected_steps.append(s)
        
        
    
    return (f_score(detected_steps,steps),detected_steps,precision(detected_steps,steps),recall(detected_steps,steps))


def score_illness(directory,method,cost,side,n_indiv,n_try):
    count_pathology=dict()
    score_pathology=dict()
    precision_pathology=dict()
    recall_pathology=dict()
    rec=dict()
    pre=dict()
    scores=dict()
    for i in range(1,n_indiv+1):
        for j in range(1,n_try+1):
        

            file=directory+"\\"+str(i)+"-"+str(j)+".csv"
            if (str(i)+"-"+str(j)+".csv" in listdir(directory)):
                data=data_acquisition(file)
                meta_data=data[1]
                type_illness=meta_data["PathologyGroup"]
                print(type_illness)
                if (not(type_illness in score_pathology)):
                    score_pathology[type_illness]=0
                    recall_pathology[type_illness]=0
                    precision_pathology[type_illness]=0
                    count_pathology[type_illness]=0
                
                count_pathology[type_illness]+=1
                res=process_data(file,method,cost,side)
                score_pathology[type_illness]+=res[0]
                scores[file]=res[0]
                
                rec[file]=res[3]
                pre[file]=res[2]
                precision_pathology[type_illness]+=res[2]
                recall_pathology[type_illness]+=res[3]
                print(res[0])
            else:
                break
    for i in score_pathology:
        score_pathology[i]=score_pathology[i]/count_pathology[i]
        precision_pathology[i]=precision_pathology[i]/count_pathology[i]
        recall_pathology[i]=recall_pathology[i]/count_pathology[i]
    return(score_pathology,precision_pathology,recall_pathology,scores,pre,rec)

results=score_illness("C:\\Users\\Yassine\\Desktop\\GaitData",rpt.BottomUp,MyCost(),'right',230,15)
            

Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\1-1.csv
0.9080047789725207
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\1-2.csv
0.9333333333333333
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\1-3.csv
0.9523809523809523
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\1-4.csv
0.9795918367346939
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\1-5.csv
0.9555325749741469
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\2-1.csv
0.9090909090909091
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\2-2.csv
0.9387755102040816
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\2-3.csv
0.9361702127659575
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\3-1.csv
0.923076923076923
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\3-2.csv
0.842911877394636
Healthy
initiating pro

0.8653846153846153
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\17-4.csv
0.9128630705394191
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\18-1.csv
0.9600000000000001
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\18-2.csv
0.8764940239043824
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\18-3.csv
0.9166666666666666
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\18-4.csv
0.9166666666666666
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\19-1.csv
0.7410926365795725
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\19-2.csv
0.7761194029850746
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\19-3.csv
0.8406304728546409
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\20-1.csv
0.9600000000000001
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\20-2.csv
0.9166666666666666
Heal

0.8825806451612903
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\38-4.csv
0.8323699421965318
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\38-5.csv
0.8323699421965318
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\38-6.csv
0.9080047789725207
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\38-7.csv
0.8450704225352113
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\38-8.csv
0.8814432989690723
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\38-9.csv
0.9767441860465117
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\38-10.csv
0.8767908309455588
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\39-1.csv
0.9166666666666666
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\39-2.csv
1.0
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\39-3.csv
0.9767441860465117
O

0.9035667107001321
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\57-2.csv
0.9655172413793104
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\57-3.csv
1.0
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\57-4.csv
0.923076923076923
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\57-5.csv
0.972972972972973
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\58-1.csv
0.8939828080229226
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\58-2.csv
0.9258160237388724
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\59-1.csv
0.9433962264150945
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\59-2.csv
0.9361702127659575
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\60-1.csv
0.7508532423208191
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\60-2.csv
0.8939828080229226
Healthy
initiating pro

0.863481228668942
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\78-2.csv
0.914657980456026
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\78-3.csv
0.9666666666666666
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\79-1.csv
0.975609756097561
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\79-2.csv
0.9500000000000001
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\79-3.csv
0.9166666666666666
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\80-1.csv
0.9166666666666666
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\80-2.csv
0.9565217391304348
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\80-3.csv
0.9128630705394191
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\80-4.csv
0.9696969696969697
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\80-5.csv
0.93333333333

...

0.8825806451612903
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\134-10.csv
0.830393487109905
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\134-11.csv
0.9090909090909091
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\134-12.csv
0.9069212410501194
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\134-13.csv
0.9080047789725207
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\134-14.csv
0.9110629067245118
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\135-1.csv
0.9071274298056156
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\135-2.csv
0.9696969696969697
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\136-1.csv
0.975609756097561
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\137-1.csv
1.0
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\137-2.csv
1.0
Healthy
initiating

0.9090909090909091
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\153-1.csv
0.9795918367346939
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\153-2.csv
0.9795918367346939
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\154-1.csv
0.9047619047619047
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\154-2.csv
0.8755364806866952
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\154-3.csv
0.7706422018348624
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\154-4.csv
0.8932676518883416
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\154-5.csv
0.9500000000000001
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\155-1.csv
0.9361702127659575
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\155-2.csv
0.927536231884058
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\1

0.962962962962963
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\173-2.csv
0.9198606271777002
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\173-3.csv
0.8450704225352113
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\173-4.csv
0.9309462915601023
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\173-5.csv
0.9285714285714286
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\174-1.csv
0.9500000000000001
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\174-2.csv
0.9500000000000001
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\174-3.csv
0.9743589743589743
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\174-4.csv
0.975609756097561
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\174-5.csv
0.9473684210526316
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\175-1.csv
0.9600000000000001
Healthy


0.9583333333333334
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\193-1.csv
0.900990099009901
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\193-2.csv
1.0
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\193-3.csv
0.9333333333333333
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\193-4.csv
0.9258160237388724
Neurological
initiating process for:  C:\Users\Yassine\Desktop\GaitData\193-5.csv
0.962962962962963
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\194-1.csv
0.9333333333333333
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\194-2.csv
0.9655172413793104
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\194-3.csv
0.9354120267260579
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\194-4.csv
0.967741935483871
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\194-5.csv
1.0
Orthopedic
initiating process

0.962962962962963
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\213-4.csv
0.9258160237388724
Healthy
initiating process for:  C:\Users\Yassine\Desktop\GaitData\213-5.csv
0.962962962962963
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\214-1.csv
0.9216867469879517
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\214-2.csv
1.0
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\214-3.csv
0.9258160237388724
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\215-1.csv
0.9032258064516129
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\215-2.csv
0.896551724137931
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\215-3.csv
0.861878453038674
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\215-4.csv
0.9696969696969697
Orthopedic
initiating process for:  C:\Users\Yassine\Desktop\GaitData\215-5.csv
0.896551724137931
Orthopedic
initia

In [12]:
## mean f_score per pathology group
print(results[0])
print(results[1])
print(results[2])

{'Neurological': 0.9083650954998987, 'Healthy': 0.9086802918395857, 'Orthopedic': 0.9301559175734249}
{'Neurological': 0.9545800511910381, 'Healthy': 0.9470677354756638, 'Orthopedic': 0.9607115984056361}
{'Neurological': 0.8688112044986568, 'Healthy': 0.8756381074587803, 'Orthopedic': 0.9038394922921337}


In [10]:
## f_score per signal processed
results[1]

{'C:\\Users\\Yassine\\Desktop\\GaitData\\1-1.csv': 0.9322974472807991,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\1-2.csv': 0.9787234042553191,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\1-3.csv': 0.9767441860465117,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\2-1.csv': 0.888888888888889,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\2-2.csv': 0.9387755102040816,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\2-3.csv': 0.9361702127659575,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\3-1.csv': 0.923076923076923,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\3-2.csv': 0.842911877394636,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\3-3.csv': 0.8319327731092436,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\4-1.csv': 0.9714285714285714,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\4-2.csv': 0.9714285714285714,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\4-3.csv': 0.9071274298056156,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\5-1.csv': 0.9279086366880799,
 'C:\\Users\\Yassine\\Desktop\\GaitData\\5-2.csv': 0.9518599562363238,
 'C:\\Use

In [13]:
## mean f_score
print(np.asarray([*results[3].values()]).mean())
print(np.asarray([*results[4].values()]).mean())
print(np.asarray([*results[5].values()]).mean())

0.9136901599066394
0.9543440723801363
0.8788132497512788
